# Testing diffusion problem format

Grid and parameters

In [1]:
struct Grid
    xmax::Float64
    ymax::Float64

    nx::Int64
    ny::Int64
    
    Δx::Float64
    Δy::Float64

    xRange::Array{Float64,1}
    yRange::Array{Float64,1}

    function Grid(xmax, ymax, nx, ny)
        Δx = 2*xmax/(nx-1)
        Δy = 2*ymax/(ny-1)

        xRange = (-xmax:Δx:xmax)
        yRange = (-ymax:Δy:ymax)

        return new(xmax, ymax, nx, ny, Δx, Δy, xRange, yRange)
    end
end
struct Parameters
    ν::Float64
    σ::Float64
    tspan::Tuple{Float64,Float64}
end

Model setup

In [2]:
abstract type FluidModel end
mutable struct DiffusionModel{T<:Union{Function,Nothing}} <: FluidModel
    u0::Array{Float64,2}
    u_analytic::T
    
    grid::Grid
    parameters::Parameters
    
    Δt::Float64
    t::Array{Float64,1}
    
    function DiffusionModel(u0, u_analytic::T, grid, parameters) where T<:Union{Function,Nothing}
        
        Δt = parameters.σ*grid.Δx*grid.Δy/parameters.ν
        t = parameters.tspan[1]:Δt:parameters.tspan[2]
        
        return new{T}(u0, u_analytic, grid, parameters, Δt ,t)
    end
end
    #function DiffusionModel(grid, parameters)
    #    u = zeros(grid.ny, grid.nx)
    #    u_analytic = nothing

    #    Δt, t = calculate_time(grid, parameters)
        
    #    return new{Nothing}(u, u_analytic, grid, parameters, Δt ,t)
    #end
    #function DiffusionModel(u0, grid, parameters)
    #    u_analytic = nothing

    #    Δt, t = calculate_time(grid, parameters)
        
    #    return new{Nothing}(u0, u_analytic, grid, parameters, Δt ,t)
    #end
    #function DiffusionModel(u0, u_analytic, grid, parameters)

    #    Δt, t = calculate_time(grid, parameters)
        
    #    return new{Function}(u0, u_analytic, grid, parameters, Δt ,t)
    #end
#end
#function calculate_time(grid::Grid, parameters::Parameters)
#    Δt = parameters.σ*grid.Δx*grid.Δy/parameters.ν
#    t = parameters.tspan[1]:Δt:parameters.tspan[2]
#    return Δt, t
#end
function DiffusionModel(grid, parameters)
    u = zeros(grid.ny, grid.nx)
    u_analytic = nothing

    return DiffusionModel{Nothing}(u, u_analytic, grid, parameters)
end
function DiffusionModel(u0, grid, parameters)
    u_analytic = nothing

    return DiffusionModel{Nothing}(u0, u_analytic, grid, parameters)
end
function DiffusionModel(u0, u_analytic, grid, parameters)

    return DiffusionModel{Function}(u0, u_analytic, grid, parameters)
end

DiffusionModel

Test setup of model

In [ ]:
#dump(DiffusionModel(copy(mod_test.u0), sqrt, grid_test, param_test))
DiffusionModel(zeros(grid_test.ny, grid_test.nx), u_analytic::T, grid, parameters)

In [3]:
grid_test = Grid(1., 2., 4, 6)
param_test = Parameters(0.2, 0.00025, (0., 1.))
mod_test = DiffusionModel(grid_test, param_test)
mod_test.u0[2,2] = 10
dump(mod_test)

LoadError: MethodError: no method matching DiffusionModel{Nothing}(::Array{Float64,2}, ::Nothing, ::Grid, ::Parameters)

Cache

In [ ]:
abstract type DiffusionCache end
mutable struct ExplicitDiffusionCache <: DiffusionCache
    u::Array{Float64,2}
    uprev::Array{Float64,2}
    t::Float64
end
mutable struct CNDiffusionCache <: DiffusionCache
    u::Array{Float64,2}
    uprev::Array{Float64,2}
    rhs::Array{Float64,2}
    a1::Array{Float64,2}
    a2::Array{Float64,2}
    t::Float64
end
mutable struct ADIDiffusionCache <: DiffusionCache
    u::Array{Float64,2}
    uhalf::Array{Float64,2}
    uprev::Array{Float64,2}
    rhs::Array{Float64,2}
    a1x::Array{Float64,2}
    a1y::Array{Float64,2}
    a2x::Array{Float64,2}
    a2y::Array{Float64,2}
    t::Float64
end

Methods

In [ ]:
abstract type DiffusionMethod end
struct ExplicitDiffusion <: DiffusionMethod end
struct CrankNickolsonDiffusion <: DiffusionMethod end
struct ADIDiffusion <: DiffusionMethod end

In [ ]:
function perform_step!(model::DiffusionModel, method::ExplicitDiffusion, cache::ExplicitDiffusionCache)
    u = cache.u
    un = cache.uprev
    nx = model.grid.nx
    ny = model.grid.ny
    Δx = model.grid.Δx
    Δy = model.grid.Δy
    Δt = model.Δt
    ν  = model.parameters.ν
    
    copy!(un,u)
    for j in 2:ny-1, i in 2:nx-1
        u[j,i] = un[j,i] + 
            ν*Δt/(Δx*Δx)*(un[j  ,i+1]-2*un[j,i]+un[j  ,i-1]) + 
            ν*Δt/(Δy*Δy)*(un[j+1,i  ]-2*un[j,i]+un[j-1,i  ])
    end
    
    return nothing
end
function apply_BCs!(cache)
    u = cache.u
    u[1,:]   .= 0.0
    u[end,:] .= 0.0
    u[:,1]   .= 0.0
    u[:,end] .= 0.0
    return nothing
end;

Cache initialisers

In [ ]:
init_cache(model, method::ExplicitDiffusion) = 
    ExplicitDiffusionCache(copy(model.u0), copy(model.u0), copy(mod_test.parameters.tspan[1]));

Problem = model + method + (cache)

In [ ]:
abstract type FluidProblem end
mutable struct DiffusionProblem{MethodType<:DiffusionMethod, CacheType<:DiffusionCache} <: FluidProblem
    model::DiffusionModel
    method::MethodType
    cache::CacheType
end
function DiffusionProblem(model, M::MethodType) where MethodType
    cache = init_cache(model, M)
    CacheType = typeof(cache)
    return DiffusionProblem{MethodType, CacheType}(model, M, cache)
end;

In [ ]:
abstract type FluidSolution end
mutable struct DiffusionSolution{T<:Union{Array{Float64,2},Nothing}} <: FluidSolution
    u::Array{Float64,2}
    
    u_analytic::T
    errors::T
    
    prob::DiffusionProblem
    retcode::Bool
end
function DiffusionSolution(u::Array{Float64,2}, prob::DiffusionProblem, retcode::Bool)
    return DiffusionSolution(u, nothing, nothing, prob, retcode)
end;

Solve problem

In [ ]:
function solve(prob::DiffusionProblem)
    sol = solve!(prob)
    return sol
end
function solve!(prob::DiffusionProblem)
    while prob.cache.t < prob.model.parameters.tspan[2]
        timestep!(prob)
    end
    #return DiffusionSolution(prob.cache.u, prob.cache.u, prob.cache.u, prob, true)
    return DiffusionSolution(prob.cache.u, prob, true)
    #return prob.cache.u
end
function timestep!(prob::DiffusionProblem)
    #println("t = ", prob.cache.t)
    perform_step!(prob.model, prob.method, prob.cache)
    apply_BCs!(prob.cache)
    prob.cache.t += prob.model.Δt
    return nothing
end;

Initialise problem object

In [ ]:
prob_test = DiffusionProblem(mod_test, ExplicitDiffusion())
dump(prob_test)

Try solving

In [ ]:
sol_test = solve(prob_test);

In [ ]:
using BenchmarkTools
@benchmark solve(prob_test)

In [ ]:
prob_test.model.u0

In [ ]:
prob_test.cache.u

In [ ]:
sol_test.u